# Checkpoint selection with ModelCheckpoint classes

This notebook shows how to replace the checkpoint selection helpers in `mycelia/shared/checkpoint_helper.py`
(`get_sorted_checkpoints`, `get_resume_info`, `start_model_from`) with the class-based flow in
`mycelia/shared/checkpoints.py`.


## Imports

In [1]:
from __future__ import annotations

from pathlib import Path

import bittensor as bt

from mycelia.shared.checkpoints import (
    ModelCheckpoint,
    ModelCheckpoints,
    ChainCheckpoint,
    ChainCheckpoints,
    Checkpoints,
    build_local_checkpoints,
    select_best_checkpoint,
    build_local_checkpoint
)


In [2]:
miner_wallet = bt.Wallet(name = 'miner', hotkey = 'hk1')

## 2b) Sign and verify a ModelCheckpoint


In [3]:
import bittensor as bt

ckpt = build_local_checkpoint(Path("/home/isabella/crucible/subnet-MoE/checkpoints/miner/miner/hk1/foundation/globalver_553_inneropt_60"))
ckpt.hotkey = miner_wallet.hotkey.ss58_address

signature = ckpt.sign_hash(wallet=miner_wallet)
print("signature", signature)
print("signature_verified_after_sign", ckpt.signature_verified)
print("verify_signature", ckpt.verify_signature())
print("active", ckpt.active())
print("model_hash", ckpt.model_hash)
print("verify_model_hash", ckpt.verify_hash())

signature c801764b4882d7bf9d629f2cab3c925468e7eaf867dde4e77b371b21d5bd201f5ead8a97a1eacb9b1ee729312af9f9678ddba228d44359fa6dc401c49f131689
signature_verified_after_sign True
verify_signature True
active False
model_hash ab3b5331a7135ed50d0f182d026e60abdb3646fd51bcf8a3
verify_model_hash True


In [4]:
len(signature)

128

In [13]:
import bittensor as bt

ckpt = build_local_checkpoint(Path("/home/isabella/crucible/subnet-MoE/checkpoints/miner/miner/hk1/foundation/globalver_553_inneropt_60"))
ckpt.hotkey = miner_wallet.hotkey.ss58_address

signature = ckpt.sign_hash(wallet=miner_wallet)
print("signature", signature)
print("signature_verified_after_sign", ckpt.signature_verified)
print("verify_signature", ckpt.verify_signature())
print("active", ckpt.active())
print("model_hash", ckpt.model_hash)
print("verify_model_hash", ckpt.verify_hash())

signature rALfiOHOCpUJM0iHx3YNLC2ueJQ8mh1hoA44IjHsZ1dYZrneJ7M1qpKGfhb98exsQasfhdRWLwJ5sioAmyq6hA
signature_verified_after_sign True
verify_signature True
active False
model_hash ab3b5331a7135ed50d0f182d026e60abdb3646fd51bcf8a3
verify_model_hash True


In [5]:
len("rALfiOHOCpUJM0iHx3YNLC2ueJQ8mh1hoA44IjHsZ1dYZrneJ7M1qpKGfhb98exsQasfhdRWLwJ5sioAmyq6hA")

86

## 1) Build local checkpoints from a folder

This replaces `get_sorted_checkpoints` and the local part of `get_resume_info`.

In [ ]:
local_checkpoints = build_local_checkpoints(Path("/path/to/miner/checkpoints"), role="miner")
local_checkpoints.ordered()[:3]


## 2) (Optional) Build chain checkpoints

This replaces chain checkpoint selection logic. Build a list of `ValidatorChainCheckpoint` objects
from on-chain submissions. Verification flags are optional unless you plan to use them in `active()`.

In [ ]:
def build_chain_checkpoints(raw_submissions: list[dict]) -> ValidatorChainCheckpoints:
    checkpoints: list[ValidatorChainCheckpoint] = []

    for item in raw_submissions:
        checkpoints.append(
            ValidatorChainCheckpoint(
                signed_model_hash=item.get("signed_model_hash"),
                model_hash=item.get("model_hash"),
                global_ver=int(item.get("global_ver", 0)),
                inner_opt=int(item.get("inner_opt", 0)),
                expert_group=item.get("expert_group"),
                role=item.get("role", "validator"),
                signature_required=item.get("signature_required", False),
                hash_required=item.get("hash_required", False),
                expert_group_check_required=item.get("expert_group_check_required", False),
                # Extra fields needed for signature/hash checks can go here.
                origin_hotkey_ss58=item.get("origin_hotkey_ss58"),
                target_hotkey_ss58=item.get("target_hotkey_ss58"),
                block=item.get("block"),
                current_block=item.get("current_block"),
                expires_at_block=item.get("expires_at_block"),
                expected_expert_group=item.get("expected_expert_group"),
            )
        )

    return ValidatorChainCheckpoints(checkpoints=checkpoints)


raw_submissions = [
    {
        "signed_model_hash": "deadbeef",
        "model_hash": "deadbeef",
        "global_ver": 12,
        "inner_opt": 3,
        "expert_group": 0,
        "role": "validator",
        "signature_required": False,
        "hash_required": False,
    }
]

chain_checkpoints = build_chain_checkpoints(raw_submissions)


## 3) Select the best checkpoint

This replaces the `start_model_from` decision logic using `Checkpoints.ordered()` and `active()`.

In [ ]:
best_ckpt = select_best_checkpoint(
    primary_dir=Path("/path/to/miner/checkpoints"),
    secondary_dir=Path("/path/to/validator/checkpoints"),
)
best_ckpt


## 4) Drop-in replacements for `checkpoint_helper.py`

These are small wrappers you can move into `checkpoint_helper.py` to replace the old helpers.

In [ ]:
def get_resume_info_with_classes(ckpt_dir: Path):
    local = build_local_checkpoints(ckpt_dir)
    ordered = local.ordered()
    if not ordered:
        return False, None, None
    top = ordered[0]
    return True, top, top.path


def start_model_from_with_classes(
    primary_dir: Path,
    secondary_dir: Path | None = None,
):
    best = select_best_checkpoint(primary_dir, secondary_dir)
    if best is None:
        return False, None, None
    return True, best, best.path
